# Рубежная контрольная работа

Эту работу выполнять не обязательно. Баллы набранные за работу просуммируются с баллами, которые были набраны за контрольную работу 1 и выставлены как оценка за промежуточную аттестацию "автоматом". 

Сделайте копию ноутбука

Присвойте переменной `v` значение свой табельный номер ИСУ, это нужно для определения вашего варианта. Запустите код в следующей ячейке.

In [ ]:
from datetime import datetime
current_datetime = datetime.now()
print(current_datetime)
v = 278386 # по умолчанию вписан мой номер

2020-12-21 10:30:52.913015


# Общее задание

Нужно решить задачу бинарной классификации, предварительно построив признаковое описание объектов на основе нескольких таблиц. 

Целевая переменная - пол клиента.

В качестве модели нужно использовать нейронную сеть, которую нужно строить с помощью `keras` или `torch` на выбор студента.

# Данные

Для всех последующих заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **transactions, tr_mcc_codes и gender_train** из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк.

Эти данные уже использовались в работе 2. Описание данных тут: https://drive.google.com/file/d/13sn8orjkoGfO1SVZYByXNmZQYFRfmOdp/view?usp=sharing

Скачать данные отсюда: https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L?usp=sharing

## Задание 1

В задании требуется на основе нескольких таблиц с данными сделать признаковое описание объектов. 

Объектами являются клиенты. Клиенты идентифицируются с помощью `customer_id`, которые есть в таблицах **transactions** и **gender_train**. В качестве призаков нужно использовать даннее по категориям транзаций: ```mcc_code``` — mcc-код транзакции есть в таблицах **transactions** и **tr_mcc_codes**


Для выбора варианта, запустите код в следующей ячейке



In [ ]:
print ('Список категорий')
feature_list = ['Одежда (также нужно искать "одежды")', 'Аптеки', 'Ценные бумаги', 'Магазины косметики и Салоны красоты', 'Образовательные услуги (а также школы, университеты)',
                'Алименты', 'Прокат автомобилей', 'Автозапчасти, Автошины и Станции техобслуживания (а также СТО)', 'Видеоигры (искать надо по "видеоигр")',
                'Отели', 'Программирование', 'Ремонт', 'Предсказатели будущего', 'Ювелирные изделия']
if (int(str(v)[4]) % 2) == 0:
  print (feature_list[::2])
else:
  print (feature_list[::-2])


if (int(str(v)[5]) % 2) == 0:
  print ('В качестве значений признаков используйте количество транзакций с тратами и количество транзакций с поступлениями')
else: 
  print ('В качестве значений признаков используйте сумму по транзакциям с тратами и сумму по транзакциям с поступлениями')

Список категорий
['Одежда (также нужно искать "одежды")', 'Ценные бумаги', 'Образовательные услуги (а также школы, университеты)', 'Прокат автомобилей', 'Видеоигры (искать надо по "видеоигр")', 'Программирование', 'Предсказатели будущего']
В качестве значений признаков используйте количество транзакций с тратами и количество транзакций с поступлениями


Каждая категория может быть связана с одним или несколькими mcc-кодами. Например, вот несколько mcc-кодов по категории "Одежда": 
* 5621; Готовая женская одежда
* 5641; Детская одежда
* 5651; Одежда для всей семьи

Таким образом получится 14 признаков, то есть семь категорий, каждая из которых даст два значения: траты и поступления. Например, категория "Одежда" даёт признаки "Одежда траты" и "Одежда поступления".

Сделайте нормализацию значений признаков так, чтобы все означения менялись от 0 до 1. 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
transactions = pd.read_csv(filepath_or_buffer = "transactions.csv",
            sep = ',',
            nrows = 1_000_000)
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN


In [ ]:
gender_train = pd.read_csv(filepath_or_buffer = "gender_train.csv",
            sep = ',')
gender_train.head()

,customer_id,gender
0,10928546,1
1,69348468,1
2,61009479,0
3,74045822,0
4,27979606,1


In [ ]:
codes = pd.read_csv('tr_mcc_codes.csv', sep=';')
codes.head()

,mcc_code,mcc_description
0,742,Ветеринарные услуги
1,1711,"Генеральные подрядчики по вентиляции, теплосна..."
2,1731,Подрядчики по электричеству
3,1799,"Подрядчики, специализированная торговля — нигд..."
4,2741,Разнообразные издательства/печатное дело


In [ ]:
# Список категорий
# ['Одежда (также нужно искать "одежды")', 'Ценные бумаги', 'Образовательные услуги (а также школы, университеты)', 'Прокат автомобилей', 'Видеоигры (искать надо по "видеоигр")', 'Программирование', 'Предсказатели будущего']
# В качестве значений признаков используйте количество транзакций с тратами и количество транзакций с поступлениями

In [ ]:
types = ('одежд', 'ценные бумаги', 'образов', 'школ', 'универ' 'прокат', 'видеоигр', 'программ', 'предсказат')

In [ ]:
d = {
    'customer' : np.array(gender_train.customer_id),
     'clothes+' : np.zeros(len(gender_train)), 'clothes-' : np.zeros(len(gender_train)), 
     'papers+': np.zeros(len(gender_train)), 'papers-': np.zeros(len(gender_train)), 
     'education+': np.zeros(len(gender_train)), 'education-': np.zeros(len(gender_train)), 
     'rent+' : np.zeros(len(gender_train)), 'rent-' : np.zeros(len(gender_train)), 
     'videogames+' : np.zeros(len(gender_train)), 'videogames-' : np.zeros(len(gender_train)), 
     'programming+' : np.zeros(len(gender_train)), 'programming-' : np.zeros(len(gender_train)), 
     'predictors+' : np.zeros(len(gender_train)), 'predictors-' : np.zeros(len(gender_train)), 
     'gender':np.array(gender_train.gender)}

data = pd.DataFrame(data=d)
data.head()

,customer,clothes+,clothes-,papers+,papers-,education+,education-,rent+,rent-,videogames+,videogames-,programming+,programming-,predictors+,predictors-,gender
0,10928546,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,69348468,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,61009479,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,74045822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,27979606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
for i, row in transactions.iterrows() :
  element = codes[codes.mcc_code == row.mcc_code].mcc_description.values[0].lower()
  sum = row.amount

  if i % 100_000 == 0:
    print(i)

  if element.startswith(types):
    if element.startswith('одежд'):
      if(sum > 0):
        data.loc[data['customer'] == row.customer_id, 'clothes+'] = data[data.customer == row.customer_id]['clothes+'] + 1
      else:
        data.loc[data['customer'] == row.customer_id, 'clothes-'] = data[data.customer == row.customer_id]['clothes-'] + 1
      
    elif element.startswith('ценные бумаги'):
      if sum > 0:
        data.loc[data['customer'] == row.customer_id, 'papers+'] = data[data['customer'] == row.customer_id]['papers+'] + 1
      else:
        data.loc[data['customer'] == row.customer_id, 'papers-'] = data[data['customer'] == row.customer_id]['papers-'] + 1

    elif element.startswith('образов') or element.startswith('школ') or element.startswith('универ'):
      if sum > 0:
        data.loc[data['customer'] == row.customer_id, 'education+'] = data[data['customer'] == row.customer_id]['education+'] + 1
      else:
        data.loc[data['customer'] == row.customer_id, 'education-'] = data[data['customer'] == row.customer_id]['education-'] + 1

    elif element.startswith('прокат'):
      if sum > 0:
        data.loc[data['customer'] == row.customer_id, 'rent+'] = data[data['customer'] == row.customer_id]['rent+'] + 1
      else:
        data.loc[data['customer'] == row.customer_id, 'rent-'] = data[data['customer'] == row.customer_id]['rent-'] + 1

    elif element.startswith('видеоигр'):
      if sum > 0:
        data.loc[data['customer'] == row.customer_id, 'videogames+'] = data[data['customer'] == row.customer_id]['videogames+'] + 1
      else:
        data.loc[data['customer'] == row.customer_id, 'videogames'] = data[data['customer'] == row.customer_id]['videogames-'] + 1
      
    elif element.startswith('програм'):
      if sum > 0:
        data.loc[data['customer'] == row.customer_id, 'programming+'] = data[data['customer'] == row.customer_id]['programming+'] + 1
      else:
        data.loc[data['customer'] == row.customer_id, 'programming-'] = data[data['customer'] == row.customer_id]['programming-'] + 1

    elif element.startswith('предсказат'):
      if sum > 0:
        data.loc[data['customer'] == row.customer_id, 'predictors+'] = data[data['customer'] == row.customer_id]['predictors+'] + 1
      else:
        data.loc[data['customer'] == row.customer_id, 'predictors-'] = data[data['customer'] == row.customer_id]['predictors-'] + 1

0
100000
200000
300000
400000
500000
600000
700000
800000
900000


In [ ]:
data_copy = data.copy()

In [ ]:
# удалим пустые строки
# и проверим новый 
# шейп

In [ ]:
counter = 0
for i, row in data_copy.iterrows():
  if row[1:len(row)-1].sum() == 0:
    counter = counter + 1
    data_copy = data_copy.drop(i, axis=0)
counter

7983

In [ ]:
counter = 0
for i, row in data_copy.iterrows():
  if row[1:len(row)-1].sum() == 0:
    counter = counter + 1
counter

0

In [ ]:
data_copy.shape

(417, 16)

In [ ]:
data_copy.head()

,customer,clothes+,clothes-,papers+,papers-,education+,education-,rent+,rent-,videogames+,videogames-,programming+,programming-,predictors+,predictors-,gender
16,34174139,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
31,16422350,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
44,10493682,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
45,79496375,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
46,27868636,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


# Задание 2

Модель - многослойная нейронная сеть с двумя скрытыми слоями и с dropout. Количество нейронов в каждом слое выбираете самостоятельно.
Функции активации, метод оптимизации, скорость обучения, вероятность dropout - выбираете самостоятельно

Модель нужно строить с помощью keras или torch

Нужно отдельно перечислить все гиперпараметры и их значения вот в таком формате (значения приведены для примера, у вас могут быть другие):

* количество эпох: 5
* скорость обучения: 0.0001
* функция активация: ReLU
* и т.д.

Настроить параметры модели.

In [ ]:
import torch
import torchvision

from sklearn import preprocessing
from sklearn.model_selection import train_test_split 

from keras.optimizers import Adam
from keras.layers import Dropout
from keras.optimizers import SGD
from sklearn.metrics import classification_report

In [ ]:
%tensorflow_version 1.x
# Импортируем сам keras
import keras
# Последовательный тип модели
from keras.models import Sequential
# Импортируем полносвязный слой, слои активации и слой, превращающий картинку в вектор
from keras.layers import Dense, Activation, Flatten
# Импортируем датасеты, чтобы вытащить оттуда нужные нам данные
import keras.datasets

import numpy as np
from matplotlib import pyplot as plt

# Эти библиотеки отключают лишние предупреждения от библиотек, в частности,
# tensorflow, чтобы не засорять вывод наших результатов
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import warnings
warnings.filterwarnings("ignore")

# keras является оболочкой для более низкоуровневых библиотек, 
# в нашем случае мы используем tensorflow, у которого на данный момент 
# поддерживаются две версии, установим первую
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_copy.drop(["gender", "customer"], axis=1), data_copy.gender, train_size = 0.7)

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

y_train=keras.utils.to_categorical(y_train,2)
# y_test=keras.utils.to_categorical(y_test,2)

In [ ]:
# Min and max value for each row
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	return minmax

# Each row in range from 0-1
def normalize_dataset(dataset, minmax):
  for row in dataset:
    for i in range(len(row)):
      if (minmax[i][1] - minmax[i][0] != 0):
        row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [ ]:
minmax = dataset_minmax(x_train)
normalize_dataset(x_train, minmax)

In [ ]:
minmax = dataset_minmax(x_test)
normalize_dataset(x_test, minmax)

In [ ]:
x_train

array([[0.        , 0.85714286, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04761905, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.0952381 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04761905, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.61904762, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
len(x_train)

291

In [ ]:
input_size = x_train[0].shape

In [ ]:
input_size

(14,)

In [ ]:
model = Sequential()

model.add(Dense(units=40, input_shape=input_size))

model.add(Activation('relu'))

model.add(Dropout(0.2, input_shape=input_size))

model.add(Dense(units=40, input_shape=input_size))

model.add(Activation('relu'))

model.add(Dense(units=2))

model.add(Activation('softmax'))

# sgd = SGD(lr=0.1, momentum=0.3)

model.compile(loss='mean_squared_error',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=20, epochs=50)

Epoch 1/50
291/291 [==============================] - 1s 2ms/step - loss: 0.2515 - accuracy: 0.4364
Epoch 2/50
291/291 [==============================] - 0s 224us/step - loss: 0.2472 - accuracy: 0.6151
Epoch 3/50
291/291 [==============================] - 0s 223us/step - loss: 0.2442 - accuracy: 0.6186
Epoch 4/50
291/291 [==============================] - 0s 260us/step - loss: 0.2403 - accuracy: 0.6220
Epoch 5/50
291/291 [==============================] - 0s 251us/step - loss: 0.2376 - accuracy: 0.6254
Epoch 6/50
291/291 [==============================] - 0s 243us/step - loss: 0.2356 - accuracy: 0.6289
Epoch 7/50
291/291 [==============================] - 0s 241us/step - loss: 0.2346 - accuracy: 0.6392
Epoch 8/50
291/291 [==============================] - 0s 234us/step - loss: 0.2327 - accuracy: 0.6254
Epoch 9/50
291/291 [==============================] - 0s 247us/step - loss: 0.2321 - accuracy: 0.6357
Epoch 10/50
291/291 [==============================] - 0s 256us/step - loss: 0.2286 

In [ ]:
    # количество эпох: 50
    # скорость обучения: 0.001
    # функция активация: ReLU
    # оптимизатор: адам
    # batch size: 20
    # кол-во нейронов: по 40 на слой

# Задание 3


Проверить качество модели не менее чем на трёх разных метриках. Вывести эти значения.

После завершения контрольной работы, дайте ссылку на colab в отдельном followup в своей теме на Piazza. Дайте разрешение на его просмотр.

Не забудьте запустить код в последней ячейке для контроля времени выполнения.

In [ ]:
pred = model.predict_classes(x_test)

In [ ]:
print(classification_report(y_test, pred, target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       0.69      1.00      0.82        82
           1       1.00      0.16      0.27        44

    accuracy                           0.71       126
   macro avg       0.84      0.58      0.55       126
weighted avg       0.80      0.71      0.63       126



In [ ]:
from datetime import datetime
current_datetime = datetime.now()
print(current_datetime)

2020-12-21 15:01:41.957343
